In [20]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import spacy

In [21]:
df_p12 = pd.read_excel("p12_2008_2016.xlsx")

df_el = pd.read_excel("dfcleaned.xlsx")

In [22]:


# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')


# Drop rows with missing content
df_p12 = df_p12.dropna(subset=['content'])

# Convert the content to lowercase
df_p12['cleaned_content'] = df_p12['content'].str.lower()

# Remove special characters, numbers, and punctuation
df_p12['cleaned_content'] = df_p12['cleaned_content'].apply(lambda x: re.sub(r'[^a-zA-Z\sáéíóúüñÁÉÍÓÚÜÑ]', '', x))

# Tokenize the text and remove stop words
stop_words = set(stopwords.words('spanish'))
df_p12['cleaned_content'] = df_p12['cleaned_content'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

# Remove extra whitespaces
df_p12['cleaned_content'] = df_p12['cleaned_content'].apply(lambda x: ' '.join(x.split()))

# Display the cleaned DataFrame
print(df_p12[['content', 'cleaned_content']].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patricio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\patricio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                             content  \
0  Como consecuencia del impacto, el hombre que i...   
1   Por Paula Kearney, El Programa de Promoción F...   
2  Otra de las cuestiones sobre las que trabajan ...   
3   Por María del Carmen Arias * , La noción de t...   
4  Edición:   anterior   \n siguiente   , © 2000-...   

                                     cleaned_content  
0  consecuencia impacto hombre iba acompañante co...  
1  paula kearney programa promoción familiar area...  
2  cuestiones trabajan operadores calle consumo d...  
3  maría carmen arias noción toxicomanía sufrido ...  
4  edición anterior siguiente wwwpaginacomar repú...  


In [23]:
df_el["media"] = "litoral"
df_p12["media"] = "p12"

In [24]:
start_date = "2008-1-1"
end_date = "2015-12-31"
mask = (df_p12['date'] > start_date) & (df_p12['date'] <= end_date)

df_p12 = df_p12.loc[mask]
len(df_p12)

360

In [25]:
df_p12.drop(columns=["position", "day", "month", "year", "description"], inplace=True, errors="ignore")
df_el.drop(columns=["query", "subtitle"], inplace=True, errors="ignore")
            

In [26]:
print(df_p12.columns)
print(df_el.columns)

Index(['date', 'title', 'url', 'content', 'cleaned_content', 'media'], dtype='object')
Index(['date', 'title', 'url', 'content', 'cleaned_content', 'media'], dtype='object')


In [27]:
df = pd.concat([df_el, df_p12], axis=0, ignore_index=True)
df.value_counts("media")

media
litoral    527
p12        360
dtype: int64

In [28]:
df.dropna(subset=['cleaned_content'], inplace=True)

PRUEBA 2008-2011

In [29]:
end_date = "2011-12-31"
mask = (df['date'] <= end_date)

df1 = df.loc[mask]



# Descargar el modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")
# Definir las stopwords en español
stop_words_spanish = spacy.lang.es.stop_words.STOP_WORDS
stop_words_spanish.update(["año", "wwwpaginacomar", "software", "desarrollado", "gnulinux"])
# Realizar lematización en español
def lemmatize(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.text.lower() not in stop_words_spanish])

# Lematizar el contenido limpio
df1['cleaned_content'] = df1['cleaned_content'].apply(lemmatize)

# Vectorizar el contenido limpio
vectorizer = CountVectorizer(max_features=1000, lowercase=True, stop_words=stop_words_spanish)
X = vectorizer.fit_transform(df1['cleaned_content'])

# Aplicar Latent Dirichlet Allocation (LDA)
num_topics = 8  # Puedes ajustar el número de temas
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Asignar las probabilidades del tema a cada documento
df1['topic_probabilities'] = lda.transform(X).tolist()

import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "2008-2011_tsne.html")


C:\Users\patricio\AppData\Local\Temp\ipykernel_2428\2719764351.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['cleaned_content'] = df1['cleaned_content'].apply(lemmatize)
C:\Users\patricio\AppData\Local\Temp\ipykernel_2428\2719764351.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['topic_probabilities'] = lda.transform(X).tolist()
c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' 

In [30]:
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "2008-2011_tsne.html")

c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


In [31]:
start_date = "2011-12-31"
mask = (df['date'] > start_date)

df2 = df.loc[mask]



# Descargar el modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")
# Definir las stopwords en español
stop_words_spanish = spacy.lang.es.stop_words.STOP_WORDS
# Realizar lematización en español
def lemmatize(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.text.lower() not in stop_words_spanish])

# Lematizar el contenido limpio
df2['cleaned_content'] = df2['cleaned_content'].apply(lemmatize)

# Vectorizar el contenido limpio
vectorizer = CountVectorizer(max_features=1000, lowercase=True, stop_words=stop_words_spanish)
X = vectorizer.fit_transform(df2['cleaned_content'])

# Aplicar Latent Dirichlet Allocation (LDA)
num_topics = 8  # Puedes ajustar el número de temas
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Asignar las probabilidades del tema a cada documento
df2['topic_probabilities'] = lda.transform(X).tolist()

pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "2012-2016_tsne.html")


C:\Users\patricio\AppData\Local\Temp\ipykernel_2428\3968579397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['cleaned_content'] = df2['cleaned_content'].apply(lemmatize)
C:\Users\patricio\AppData\Local\Temp\ipykernel_2428\3968579397.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['topic_probabilities'] = lda.transform(X).tolist()
c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' 

In [32]:
pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "2012-2016_tsne.html")

c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


## Modelado por medio

El Litoral

In [33]:
df_el = df[df["media"] == "litoral"]

# Definir las stopwords en español
stop_words_spanish = spacy.lang.es.stop_words.STOP_WORDS
# Realizar lematización en español
def lemmatize(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.text.lower() not in stop_words_spanish])

# Lematizar el contenido limpio
df_el['cleaned_content'] = df_el['cleaned_content'].apply(lemmatize)

# Vectorizar el contenido limpio
vectorizer = CountVectorizer(max_features=1000, lowercase=True, stop_words=stop_words_spanish)
X = vectorizer.fit_transform(df_el['cleaned_content'])

# Aplicar Latent Dirichlet Allocation (LDA)
num_topics = 8  # Puedes ajustar el número de temas
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Asignar las probabilidades del tema a cada documento
df_el['topic_probabilities'] = lda.transform(X).tolist()

pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "ellitoral.html")

C:\Users\patricio\AppData\Local\Temp\ipykernel_2428\1241119130.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_el['cleaned_content'] = df_el['cleaned_content'].apply(lemmatize)
C:\Users\patricio\AppData\Local\Temp\ipykernel_2428\1241119130.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_el['topic_probabilities'] = lda.transform(X).tolist()
c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to '

Página 12

In [34]:
df_p12 = df[df["media"] == "p12"]

# Definir las stopwords en español
stop_words_spanish = spacy.lang.es.stop_words.STOP_WORDS
# Realizar lematización en español
def lemmatize(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.text.lower() not in stop_words_spanish])

# Lematizar el contenido limpio
df_p12['cleaned_content'] = df_p12['cleaned_content'].apply(lemmatize)

# Vectorizar el contenido limpio
vectorizer = CountVectorizer(max_features=1000, lowercase=True, stop_words=stop_words_spanish)
X = vectorizer.fit_transform(df_p12['cleaned_content'])

# Aplicar Latent Dirichlet Allocation (LDA)
num_topics = 8  # Puedes ajustar el número de temas
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Asignar las probabilidades del tema a cada documento
df_p12['topic_probabilities'] = lda.transform(X).tolist()

pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "p12.html")

C:\Users\patricio\AppData\Local\Temp\ipykernel_2428\2722564612.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p12['cleaned_content'] = df_p12['cleaned_content'].apply(lemmatize)
C:\Users\patricio\AppData\Local\Temp\ipykernel_2428\2722564612.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p12['topic_probabilities'] = lda.transform(X).tolist()
c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 t

## Por diario y por período

El Litoral 2008-2011

In [35]:
df_el = df[df["media"] == "litoral"]

end_date = "2011-12-31"
mask = (df_el['date'] <= end_date)

df_el = df_el.loc[mask]


# Definir las stopwords en español
stop_words_spanish = spacy.lang.es.stop_words.STOP_WORDS
# Realizar lematización en español
def lemmatize(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.text.lower() not in stop_words_spanish])

# Lematizar el contenido limpio
df_el['cleaned_content'] = df_el['cleaned_content'].apply(lemmatize)

# Vectorizar el contenido limpio
vectorizer = CountVectorizer(max_features=1000, lowercase=True, stop_words=stop_words_spanish)
X = vectorizer.fit_transform(df_el['cleaned_content'])

# Aplicar Latent Dirichlet Allocation (LDA)
num_topics = 8  # Puedes ajustar el número de temas
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Asignar las probabilidades del tema a cada documento
df_el['topic_probabilities'] = lda.transform(X).tolist()

pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "ellitoral_2008-2011.html")

c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


El Litoral 2011-2016

In [36]:
df_el = df[df["media"] == "litoral"]

start_date = "2011-12-31"
mask = (df_el['date'] > start_date)

df_el = df_el.loc[mask]


# Definir las stopwords en español
stop_words_spanish = spacy.lang.es.stop_words.STOP_WORDS
# Realizar lematización en español
def lemmatize(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.text.lower() not in stop_words_spanish])

# Lematizar el contenido limpio
df_el['cleaned_content'] = df_el['cleaned_content'].apply(lemmatize)

# Vectorizar el contenido limpio
vectorizer = CountVectorizer(max_features=1000, lowercase=True, stop_words=stop_words_spanish)
X = vectorizer.fit_transform(df_el['cleaned_content'])

# Aplicar Latent Dirichlet Allocation (LDA)
num_topics = 8  # Puedes ajustar el número de temas
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Asignar las probabilidades del tema a cada documento
df_el['topic_probabilities'] = lda.transform(X).tolist()

pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "ellitoral_2011-2016.html")

c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\patricio\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


Página12 2008-2011

In [ ]:
df_p12 = df[df["media"] == "p12"]

end_date = "2011-12-31"
mask = (df_p12['date'] <= end_date)

df_p12 = df_p12.loc[mask]

# Descargar el modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Definir las stopwords en español
stop_words_spanish = spacy.lang.es.stop_words.STOP_WORDS
# Realizar lematización en español
def lemmatize(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.text.lower() not in stop_words_spanish])

# Lematizar el contenido limpio
df_p12['cleaned_content'] = df_p12['cleaned_content'].apply(lemmatize)

# Vectorizar el contenido limpio
vectorizer = CountVectorizer(max_features=1000, lowercase=True, stop_words=stop_words_spanish)
X = vectorizer.fit_transform(df_p12['cleaned_content'])

# Aplicar Latent Dirichlet Allocation (LDA)
num_topics = 4  # Puedes ajustar el número de temas
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Asignar las probabilidades del tema a cada documento
df_p12['topic_probabilities'] = lda.transform(X).tolist()

pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "p12_2008-2011.html")

AttributeError: module 'spacy.lang' has no attribute 'es'

Página12 2011-2016

In [19]:
df_p12 = df[df["media"] == "p12"]

start_date = "2011-12-31"
mask = (df_p12['date'] > start_date)

df_p12 = df_p12.loc[mask]

# Descargar el modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Definir las stopwords en español
stop_words_spanish = spacy.lang.es.stop_words.STOP_WORDS
# Realizar lematización en español
def lemmatize(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if token.text.lower() not in stop_words_spanish])

# Lematizar el contenido limpio
df_p12['cleaned_content'] = df_p12['cleaned_content'].apply(lemmatize)

# Vectorizar el contenido limpio
vectorizer = CountVectorizer(max_features=1000, lowercase=True, stop_words=stop_words_spanish)
X = vectorizer.fit_transform(df_p12['cleaned_content'])

# Aplicar Latent Dirichlet Allocation (LDA)
num_topics = 8  # Puedes ajustar el número de temas
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Asignar las probabilidades del tema a cada documento
df_p12['topic_probabilities'] = lda.transform(X).tolist()

pyLDAvis.enable_notebook()
prueba = pyLDAvis.lda_model.prepare(lda, X, vectorizer, mds='tsne', n_jobs=1)
pyLDAvis.save_html(prueba, "p12_2011-2016.html")

NameError: name 'pyLDAvis' is not defined

In [103]:
len(df_p12)

300

In [104]:
df_p12 = df[df["media"] == "p12"]

end_date = "2011-12-31"
mask = (df_p12['date'] <= end_date)

df_p12 = df_p12.loc[mask]
len(df_p12)

60